# Setup 

## Import packages 

In [1]:
# General 
import os 
import numpy as np
import pandas as pd 

# For manipulating string objects 
import re
# for generating any necessary directories
import pathlib 
# For manipulating list objects 
import itertools 

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
# For working with sequence objects 
from Bio.Seq import Seq
# For testing restriction sites 
from Bio.Restriction import *

In [5]:
# For fetching sequences from Entrez 
from Bio import Entrez  
from Bio import SeqIO

In [6]:
# For extracting features 
from Bio.SeqFeature import SeqFeature, FeatureLocation
# For creating SeqRecord objects 
from Bio.SeqRecord import SeqRecord

# Functions 

In [7]:
import sys
# Importing module of personal functions 
sys.path.append('/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR')
from kCellReadR import * 

sys.path.append('/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/alCellREADR')
from py_modules import *

## Misc

In [8]:
speciesName = 'Mouse'
geneName = 'Cfos'

# Sequence

## Loading sequences 

In [9]:
import mygene

In [10]:
mg = mygene.MyGeneInfo()
symbolSearch = 'symbol:' + geneName
tempID_search = mg.query(geneName, scopes='symbol', fields=['ensembltranscript'], species=speciesName)['hits']

In [11]:
ensembl_transcriptIDs = return_ensemblTranscriptIDs(speciesName, geneName)
ensembl_transcriptIDs

['ENSMUST00000021674',
 'ENSMUST00000134311',
 'ENSMUST00000136122',
 'ENSMUST00000140525']

In [12]:
variantTable = table_transcriptsInfo(ensembl_transcriptIDs)  
variantTable

,TranscriptNum,TranscriptID,TranscriptName,Assembly,Type,AA_Length,Is_Canonical
0,1,ENSMUST00000021674,Fos-201,GRCm39,protein_coding,380,True
1,2,ENSMUST00000134311,Fos-202,GRCm39,protein_coding_CDS_not_defined,no protein,False
2,3,ENSMUST00000136122,Fos-203,GRCm39,protein_coding_CDS_not_defined,no protein,False
3,4,ENSMUST00000140525,Fos-204,GRCm39,protein_coding_CDS_not_defined,no protein,False


In [13]:
# Chooses canonical transcript
spliceVariant = int(variantTable[variantTable['Is_Canonical'] == 'True']['TranscriptNum'])
spliceVariant

1

In [14]:
geneName

'Cfos'

In [15]:
speciesName

'Mouse'

In [16]:
rC_exon_records, rC_intron_records, CDS, cDNA, genomic = load_referenceSequences(speciesName, geneName, spliceVariant)

In [17]:
rC_exon_records

[SeqRecord(seq=Seq('CTGTGTGTTGACAGGAGAGCCCATGCTGGAGAAGGAGTCGGCTGGGGAATGGTA...GTT'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('CTGCTCTACTTTGCCCCTTCTGCCGATGCTCTGCGCTCTGCCTCCTGACACGGT...GTC'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('CGCTTGGAGTGTATCTGTCAGCTCCCTCCTCCGATTCCGGCACTTGGCTGCAGC...TAG'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('CGCATTCAACTTAAATGCTTTTATTGACAATGTCTTGGAACAATAAGCAAACAA...CTC'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[])]

In [18]:
rC_intron_records

[SeqRecord(seq=Seq('ctgaaacaaaacaaaaagaggaaagcggaggtgagcgaggaggttccagacaca...cac'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('ctatagaagaaaaggaccaacattcagttaaggttgtctagtgtcacctccatt...cac'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('ctgcatagggagagatataagaaagcataagacacgggcatgctcaaggaccct...tac'), id='<unknown id>', name='<unknown name>', description='<unknown description>', dbxrefs=[])]

In [19]:
len(CDS[0].seq)

1143

In [20]:
len(genomic[0].seq)

3384

## Selecting sensor 

In [21]:
import random 
n = random.random() 
print(n)

0.7381336749593435


In [22]:
len(genomic[0])

3384

In [23]:
len(CDS[0])

1143

In [24]:
# Print lenth exons 
for exon in rC_exon_records:
    print(len(str(exon.seq)))

291
252
108
1457


In [25]:
max([len(exon) for exon in rC_exon_records])

1457

In [26]:
len(rC_exon_records)

4

In [27]:
rC_CDS = [CDS[0].reverse_complement()]
rC_genomic = [genomic[0].reverse_complement()]

In [28]:
##### Initial if want to check variable length sesRNAs 
for i in range(192, 300):
    if(i%3 == 0):
        print(i)

192
195
198
201
204
207
210
213
216
219
222
225
228
231
234
237
240
243
246
249
252
255
258
261
264
267
270
273
276
279
282
285
288
291
294
297


In [50]:
# Longer the sesRNA ... the better it binds ... but the higher likelihood to introduce stop codons and ATGs 
# Want around 60 bp distance between multiple TAGs ... 
sesRNA_length = 200
targetChoice = 'cds' # exon, cds, genomic
parameters = parameters_sesRNA(speciesName, geneName,  spliceVariant, sesRNA_length, 1, 4, 'Upstream central', 40, 70, 20, 10)

In [51]:
if targetChoice == 'exon':
    target = rC_exon_records 
    test = cDNA
elif targetChoice == 'cds':
    target = rC_CDS
    test = cDNA
elif targetChoice == 'genomic':
    target = rC_genomic
    test = genomic
elif targetChoice == 'custom':
    target = Seq('TGA')
    test = genomic

In [52]:
parameters.print_parameters()

[instance attributes]
species = Mouse
gene = Cfos
spliceVariant = 1
length = 200
num_inF_TGG = 1
num_inF_Stop = 4
inF_ATG = Upstream central
minGC = 40
maxGC = 70
nearCenter = 20
fromStop = 10


In [53]:
sesRNA_length/2

100.0

In [54]:
# target = Seq()

In [55]:
all_sesRNAs, all_sequenceMetrics, all_sesRNA_objs = generate_all_sesRNAs(target, test, parameters, variantTable['Type'])
len(all_sequenceMetrics)

1


/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/kCellReadR/sesRNAs.py:92: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_sequenceMetrics = all_sequenceMetrics.append(temp_sequenceMetrics)


178

In [56]:
all_sequenceMetrics

,sesNum,SeqNum,ExonFrac,ExonProtFrac,CDSFrac,CDNAFrac,StartSeq,StopSeq,firstTGG,centralTGG,second_cTGG,numTGG,numTTGG,numTGGA,numTTGGA,numATG,numStop,gcCont
0,1,1,1/4,1/1,1/1,1/4,0,200,87,87,NA,1,1,0,0,0,1,61.5
1,2,1,1/4,1/1,1/1,1/4,3,203,84,84,NA,1,1,0,0,0,1,62.5
2,3,1,1/4,1/1,1/1,1/4,6,206,81,81,NA,1,1,0,0,0,1,62.5
3,4,1,1/4,1/1,1/1,1/4,156,356,117,117,NA,1,0,0,0,0,2,58.5
4,5,1,1/4,1/1,1/1,1/4,159,359,114,114,NA,1,0,0,0,0,2,58.5
5,6,1,1/4,1/1,1/1,1/4,162,362,111,111,NA,1,0,0,0,0,2,58.5
6,7,1,1/4,1/1,1/1,1/4,165,365,108,108,NA,1,0,0,0,0,2,58.5
7,8,1,1/4,1/1,1/1,1/4,168,368,105,105,NA,1,0,0,0,0,2,57.5
8,9,1,1/4,1/1,1/1,1/4,171,371,102,102,NA,1,0,0,0,0,2,58.0
9,10,1,1/4,1/1,1/1,1/4,174,374,99,99,NA,1,0,0,0,0,2,58.0


# Secondary structure 

## Working 

In [57]:
generate_RNApred(all_sesRNAs, all_sequenceMetrics, geneName, 1)

,sesNum,SeqNum,ExonFrac,ExonProtFrac,CDSFrac,CDNAFrac,StartSeq,StopSeq,firstTGG,centralTGG,second_cTGG,numTGG,numTTGG,numTGGA,numTTGGA,numATG,numStop,gcCont,mfe
0,1,1,1/4,1/1,1/1,1/4,0,200,87,87,NA,1,1,0,0,0,1,61.5,-78.2
1,2,1,1/4,1/1,1/1,1/4,3,203,84,84,NA,1,1,0,0,0,1,62.5,-77.0
2,3,1,1/4,1/1,1/1,1/4,6,206,81,81,NA,1,1,0,0,0,1,62.5,-78.9
3,4,1,1/4,1/1,1/1,1/4,156,356,117,117,NA,1,0,0,0,0,2,58.5,-70.3
4,5,1,1/4,1/1,1/1,1/4,159,359,114,114,NA,1,0,0,0,0,2,58.5,-72.8
5,6,1,1/4,1/1,1/1,1/4,162,362,111,111,NA,1,0,0,0,0,2,58.5,-75.3
6,7,1,1/4,1/1,1/1,1/4,165,365,108,108,NA,1,0,0,0,0,2,58.5,-72.8
7,8,1,1/4,1/1,1/1,1/4,168,368,105,105,NA,1,0,0,0,0,2,57.5,-72.4
8,9,1,1/4,1/1,1/1,1/4,171,371,102,102,NA,1,0,0,0,0,2,58.0,-72.4
9,10,1,1/4,1/1,1/1,1/4,174,374,99,99,NA,1,0,0,0,0,2,58.0,-75.0


In [58]:
all_sequenceMetrics

,sesNum,SeqNum,ExonFrac,ExonProtFrac,CDSFrac,CDNAFrac,StartSeq,StopSeq,firstTGG,centralTGG,second_cTGG,numTGG,numTTGG,numTGGA,numTTGGA,numATG,numStop,gcCont,mfe
0,1,1,1/4,1/1,1/1,1/4,0,200,87,87,NA,1,1,0,0,0,1,61.5,-78.2
1,2,1,1/4,1/1,1/1,1/4,3,203,84,84,NA,1,1,0,0,0,1,62.5,-77.0
2,3,1,1/4,1/1,1/1,1/4,6,206,81,81,NA,1,1,0,0,0,1,62.5,-78.9
3,4,1,1/4,1/1,1/1,1/4,156,356,117,117,NA,1,0,0,0,0,2,58.5,-70.3
4,5,1,1/4,1/1,1/1,1/4,159,359,114,114,NA,1,0,0,0,0,2,58.5,-72.8
5,6,1,1/4,1/1,1/1,1/4,162,362,111,111,NA,1,0,0,0,0,2,58.5,-75.3
6,7,1,1/4,1/1,1/1,1/4,165,365,108,108,NA,1,0,0,0,0,2,58.5,-72.8
7,8,1,1/4,1/1,1/1,1/4,168,368,105,105,NA,1,0,0,0,0,2,57.5,-72.4
8,9,1,1/4,1/1,1/1,1/4,171,371,102,102,NA,1,0,0,0,0,2,58.0,-72.4
9,10,1,1/4,1/1,1/1,1/4,174,374,99,99,NA,1,0,0,0,0,2,58.0,-75.0


In [59]:
# Code block for testing against different species than sesRNA was generated against 
#test_speciesName = 'Crab-eating macaque'
test_speciesName = speciesName

# Downloading species name if test species is not same as initial species 
if test_speciesName != speciesName:
    test_ensembl_transcriptIDs = return_ensemblTranscriptIDs(test_speciesName, geneName)
    test_variantTable = table_transcriptsInfo(test_ensembl_transcriptIDs)
    # Chooses canonical transcript
    test_spliceVariant = int(test_variantTable[test_variantTable['Is_Canonical'] == 'True']['TranscriptNum'])
    test_rC_exon_records, test_C_exon_records, test_CDS, test_cDNA, test_genomic = load_referenceSequences(test_speciesName, geneName, test_spliceVariant)

In [60]:
metricsTable_higherOrder, outputRIblast = output_RIblast(all_sequenceMetrics, geneName, test_speciesName, spliceVariant, 'cDNA')
outputRIblast

/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/Output/EnsemblSeq/Mouse/Cfos_cdna-1_Mouse.fasta
RIblast ris mode has started.
Rna interaction search of query:Cfos_01 sesRNA #01 has started.
Rna interaction search of query:Cfos_01 sesRNA #01 has finished.
RIblast ris mode has finished.
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/Output/BioPython/Temp/Cfos_01.fasta
RIblast ris mode has started.
Rna interaction search of query:Cfos_02 sesRNA #02 has started.


/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/kCellReadR/higherOrder.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  useful_RIblast = useful_RIblast.append(temp_RIblast_ouput)
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/kCellReadR/higherOrder.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  useful_RIblast = useful_RIblast.append(temp_RIblast_ouput)


Rna interaction search of query:Cfos_02 sesRNA #02 has finished.
RIblast ris mode has finished.
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/Output/BioPython/Temp/Cfos_02.fasta
RIblast ris mode has started.
Rna interaction search of query:Cfos_03 sesRNA #03 has started.
Rna interaction search of query:Cfos_03 sesRNA #03 has finished.
RIblast ris mode has finished.
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/Output/BioPython/Temp/Cfos_03.fasta
RIblast ris mode has started.
Rna interaction search of query:Cfos_04 sesRNA #04 has started.


/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/kCellReadR/higherOrder.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  useful_RIblast = useful_RIblast.append(temp_RIblast_ouput)
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/kCellReadR/higherOrder.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  useful_RIblast = useful_RIblast.append(temp_RIblast_ouput)


Rna interaction search of query:Cfos_04 sesRNA #04 has finished.
RIblast ris mode has finished.
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/Output/BioPython/Temp/Cfos_04.fasta
RIblast ris mode has started.
Rna interaction search of query:Cfos_05 sesRNA #05 has started.
Rna interaction search of query:Cfos_05 sesRNA #05 has finished.
RIblast ris mode has finished.
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/Output/BioPython/Temp/Cfos_05.fasta
RIblast ris mode has started.
Rna interaction search of query:Cfos_06 sesRNA #06 has started.


...

Rna interaction search of query:Cfos_95 sesRNA #95 has finished.
RIblast ris mode has finished.
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/Output/BioPython/Temp/Cfos_95.fasta
RIblast ris mode has started.
Rna interaction search of query:Cfos_96 sesRNA #96 has started.
Rna interaction search of query:Cfos_96 sesRNA #96 has finished.
RIblast ris mode has finished.
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/Output/BioPython/Temp/Cfos_96.fasta
RIblast ris mode has started.
Rna interaction search of query:Cfos_97 sesRNA #97 has started.


/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/kCellReadR/higherOrder.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  useful_RIblast = useful_RIblast.append(temp_RIblast_ouput)
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/kCellReadR/higherOrder.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  useful_RIblast = useful_RIblast.append(temp_RIblast_ouput)


Rna interaction search of query:Cfos_97 sesRNA #97 has finished.
RIblast ris mode has finished.
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/Output/BioPython/Temp/Cfos_97.fasta
RIblast ris mode has started.
Rna interaction search of query:Cfos_98 sesRNA #98 has started.
Rna interaction search of query:Cfos_98 sesRNA #98 has finished.
RIblast ris mode has finished.
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/Output/BioPython/Temp/Cfos_98.fasta
RIblast ris mode has started.
Rna interaction search of query:Cfos_99 sesRNA #99 has started.
Rna interaction search of query:Cfos_99 sesRNA #99 has finished.
RIblast ris mode has finished.
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/Output/BioPython/Temp/Cfos_99.fasta


/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/kCellReadR/higherOrder.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  useful_RIblast = useful_RIblast.append(temp_RIblast_ouput)
/Users/kbw29/Dropbox/Research/Neurobiology_PhD/Huang/Projects/CellReadR/Code/CellREADR/kCellReadR/higherOrder.py:208: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  useful_RIblast = useful_RIblast.append(temp_RIblast_ouput)


,Accessibility Energy,Hybridization Energy,Interaction Energy,BasePair,Accessibility Energy,Hybridization Energy,Interaction Energy,BasePair
0,100.14200,-286.18,-186.03800,(0-199:1292-1093),12.41490,-26.75,-14.33510,(18-41:278-243)
0,104.95600,-288.23,-183.27400,(0-199:1289-1090),12.51560,-26.75,-14.23440,(15-38:278-243)
0,105.19200,-288.80,-183.60800,(0-199:1286-1087),12.64870,-26.75,-14.10130,(12-35:278-243)
0,10.45870,-18.57,-8.11127,(147-165:408-389),6.69457,-14.73,-8.03543,(23-35:530-509)
0,5.87373,-13.99,-8.11627,(116-124:198-190),NaN,NaN,NaN,NaN
0,112.25100,-275.15,-162.89900,(0-199:1124-925),10.19400,-18.57,-8.37599,(135-153:408-389)
0,110.33700,-276.04,-165.70300,(0-199:1121-922),9.86008,-18.57,-8.70992,(132-150:408-389)
0,9.74101,-18.57,-8.82899,(129-147:408-389),5.01115,-17.64,-12.62890,(181-199:529-511)
0,81.62950,-256.78,-175.15100,(0-199:760-561),10.24510,-18.42,-8.17488,(3-18:369-356)
0,81.16960,-255.12,-173.95000,(0-199:757-558),37.65070,-127.01,-89.35930,(99-199:658-552)


In [61]:
metricsTable_higherOrder.sort_values(' Interaction Energy', axis = 0)

,sesNum,SeqNum,ExonFrac,ExonProtFrac,CDSFrac,CDNAFrac,StartSeq,StopSeq,firstTGG,centralTGG,second_cTGG,numTGG,numTTGG,numTGGA,numTTGGA,numATG,numStop,gcCont,mfe,Accessibility Energy,Hybridization Energy,Interaction Energy,BasePair
85,86,1,0/4,0/1,1/1,4/4,473,673,117,117,NA,1,1,0,0,0,1,51.5,-61.5,95.03290,-288.27,-193.23700,(0-199:455-256)
87,88,1,0/4,0/1,1/1,4/4,479,679,111,111,NA,1,1,0,0,0,1,52.5,-61.5,95.50760,-288.16,-192.65200,(0-199:452-253)
89,90,1,0/4,0/1,1/1,4/4,485,685,105,105,NA,1,1,0,0,0,1,52.5,-57.8,95.75250,-287.87,-192.11800,(0-199:449-250)
91,92,1,0/4,0/1,1/1,4/4,491,691,99,99,195,2,2,0,0,0,1,52.0,-62.0,100.48700,-288.36,-187.87300,(0-199:446-247)
34,35,1,2/4,1/1,1/1,2/4,314,514,81,81,135,2,1,1,0,2,4,57.5,-71.2,81.52990,-267.92,-186.39000,(0-199:687-488)
0,1,1,1/4,1/1,1/1,1/4,0,200,87,87,NA,1,1,0,0,0,1,61.5,-78.2,100.14200,-286.18,-186.03800,(0-199:1292-1093)
44,45,1,2/4,1/1,1/1,2/4,342,542,36,102,48,4,0,0,0,0,0,59.5,-77.9,107.10100,-292.83,-185.72900,(0-199:527-328)
123,124,1,0/4,0/1,1/1,4/4,605,805,81,81,NA,1,1,0,0,0,0,54.5,-45.6,103.10800,-288.78,-185.67200,(0-199:947-748)
125,126,1,0/4,0/1,1/1,4/4,727,927,114,114,135,2,0,0,0,1,2,62.0,-79.2,103.81600,-288.87,-185.05400,(0-199:944-745)
121,122,1,0/4,0/1,1/1,4/4,599,799,87,87,NA,1,1,0,0,0,0,54.0,-48.1,102.74200,-287.26,-184.51800,(0-199:950-751)


# Final sesRNA check

In [ ]:
str(all_sesRNAs[2])

In [42]:
output_json = idtdna.return_complexity_score(str(all_sesRNAs[2]))

In [ ]:
output_json

In [ ]:
scores = [idtdna.return_complexity_score(str(sesRNAs))['Score'] for sesRNAs in all_sesRNAs]

In [ ]:
scores 

In [ ]:
max(scores)

In [ ]:
[scores.index(x) for x in scores if x <= 10]

## Choose sesRNA

In [68]:
metricsTable_higherOrder.sort_values(' Interaction Energy', axis = 0)[metricsTable_higherOrder['StartSeq'] > 800]

/var/folders/9l/35n9s6_s0f3bmw4yz19gz84w0000gn/T/ipykernel_81802/1151506298.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  metricsTable_higherOrder.sort_values(' Interaction Energy', axis = 0)[metricsTable_higherOrder['StartSeq'] > 800]


,sesNum,SeqNum,ExonFrac,ExonProtFrac,CDSFrac,CDNAFrac,StartSeq,StopSeq,firstTGG,centralTGG,second_cTGG,numTGG,numTTGG,numTGGA,numTTGGA,numATG,numStop,gcCont,mfe,Accessibility Energy,Hybridization Energy,Interaction Energy,BasePair
171,172,1,0/4,0/1,1/1,1/4,840,1040,42,90,42,2,0,0,0,1,0,61.0,-75.7,84.56440,-263.78,-179.21600,(0-199:792-593)
170,171,1,0/4,0/1,1/1,1/4,839,1039,6,99,96,9,1,2,0,1,3,61.0,-80.3,86.04150,-264.11,-178.06900,(0-199:795-596)
172,173,1,0/4,0/1,1/1,1/4,842,1042,3,102,96,9,1,2,0,1,3,61.5,-80.6,83.71870,-261.78,-178.06100,(0-199:789-590)
168,169,1,0/4,0/1,1/1,1/4,836,1036,9,99,102,9,1,2,0,1,3,61.0,-79.6,83.89310,-261.37,-177.47700,(0-199:801-602)
167,168,1,0/4,0/1,1/1,1/4,834,1034,48,96,48,2,0,0,0,1,0,61.0,-77.3,83.55270,-260.49,-176.93700,(0-199:804-605)
174,175,1,0/4,0/1,1/1,1/4,845,1045,0,99,93,9,1,2,0,1,3,61.0,-80.6,84.06030,-260.50,-176.44000,(0-199:783-584)
169,170,1,0/4,0/1,1/1,1/4,837,1037,45,93,45,2,0,0,0,1,0,61.0,-76.0,85.76540,-262.11,-176.34500,(0-199:798-599)
173,174,1,0/4,0/1,1/1,1/4,843,1043,39,87,39,2,0,0,0,1,0,61.5,-79.0,83.42510,-259.63,-176.20500,(0-199:786-587)
175,176,1,0/4,0/1,1/1,1/4,846,1046,36,84,36,2,0,0,0,1,0,61.5,-79.0,82.04600,-257.24,-175.19400,(0-199:763-564)
165,166,1,0/4,0/1,1/1,1/4,831,1031,51,99,51,2,0,0,0,1,0,62.0,-81.8,89.04010,-263.57,-174.53000,(0-199:810-611)


In [73]:
sesChoice = 155
chosenSequence = all_sesRNAs[sesChoice - 1]

In [74]:
scores[sesChoice - 1]

NameError: name 'scores' is not defined

In [75]:
str(chosenSequence)

'CCGCTCTGGCGTAAGCCCCAGCAGACTGGGTGGGGAGTCCGTAAGGATGGGGCGCTCTGGTCTGCGATGGGGCCACGGAGGAGACCAGAGTGGGCTGCACCAGCCACTGCAGGTCTGGGCTGGTGGAGATGGCTGTCACCGTGGGGATAAAGTTGGCACTAGAGACGGACAGATCTGCGCAAAAGTCCTGTGTGTTGACA'

In [ ]:
return_inFrame(chosenSequence, 'all')

In [ ]:
len(EcoRI.search(chosenSequence)) == 0

In [ ]:
len(AscI.search(chosenSequence)) == 0

In [ ]:
len(HindIII.search(chosenSequence)) == 0

In [ ]:
testSeq = 'CCTCCTCGCTGCCCTCGGACTTGAGGATGTCCATCTGCAGCCCTTGCCGATGCTCCATGTCCAGGTCGTCGCAGTGGGCGAAGCCCACCGCCTCCTCGTCGGTGGCCGCCTGGAAGCCCATCCTGGCGAACATGCCGCTCATCTTCGCCTGGGATTTGTGCGACACCGAGGTGGCCACGTTGGAGAGCTTGCTGCGGAGGAGGG'

In [ ]:
testSeq = 'CCTCCTCGCTGCCCTCGGACTTGAGGATGTCCATCTGCAGCCCTTGCCGATGCTCCATGTCCAGGTCGTCGCAGTGGGCGAAGCCCACCGCCTCCTCGTCGGTGGCCGCCTGGAAGCCCATCCTGGCGAACATGCCGCTCATCTTCGCCTGGGATTTGTGCGACACCGAGGTGGCCACGTTGGAGAGCTTGCTGCGGAGGAGGG'

In [ ]:
return_inFrame(Seq(testSeq), 'all')

In [ ]:
save_all_sesRNAs_DNA(all_sesRNAs, speciesName, geneName)

In [ ]:
AscI.site

# Outputing sesRNA

In [ ]:
testSeq = Seq('TGGGAGTAGTGGTGGTAATGA')
testStr = str(testSeq)

In [ ]:
testStr

In [ ]:
convert_DNA(testSeq, 1)

In [ ]:
# Test that all and numbering is working 
convert_DNA(testSeq, 3) == convert_DNA(testSeq, 'All')

In [ ]:
convert_DNA(rC_multiExon_sesRNAs[0], 'All')

In [ ]:
# Testing that continious reading frame being produced if number set to 0 
len(convert_DNA(rC_multiExon_sesRNAs[0], 0).translate(to_stop = True)) == len(rC_multiExon_sesRNAs[0])/3

In [ ]:
# Checking that right number of TGG being converted 
return_inFrame(convert_DNA(rC_multiExon_sesRNAs[0], 'All').back_transcribe(), 'numTGG')

In [ ]:
# Function for saving both original template DNA sequence and the converted RNA given list of sesRNAs Seq objects
# Could change later to make for separate output files ... have not decided yet whether to just put them together into one labeled fasta file 
def save_sesRNAs(sequences_sesRNAs, geneName, version, numConvertTGG):
    # Generating BioPython directory if does not exist 
    pathlib.Path('Output/BioPython').mkdir(parents=True, exist_ok=True)
    
    # Defining save name and description for outputs (original DNA and converted RNA)
    DNA_outputID = geneName + '_sesRNA'
    DNA_outputDescription = "sesRNA DNA original for " + geneName
    convertedRNA_outputID = geneName + '_sesRNA'
    convertedRNA_outputDescription = "sesRNA converted RNA for " + geneName
    
    # Creating empty list for storing sequences 
    outputSeqMulti_DNA = []
    outputSeqMulti_convertedRNA= []
    
    # Generating SeqRecord objects in preparation for writing FASTA files ... edited description to include number of sequence
    i = 1
    for sequence in sequences_sesRNAs:
        outputSeqMulti_DNA.append(SeqRecord(sequence, id = DNA_outputID+str(i), description = DNA_outputDescription))
        outputSeqMulti_convertedRNA.append(SeqRecord(convert_DNA(sequence, numberConvertTGG), id = convertedRNA_outputID+str(i), description = convertedRNA_outputDescription))
        i += 1
        
    # Write output fasta files 
    DNA_outputName = "Output/BioPython/" + geneName + "_sesRNA_" + version + ".fasta" 
    convertedRNA_outputName = "Output/BioPython/" + geneName + "_sesRNA_convertedRNA_" + version + ".fasta" 
    
    with open(outputName, "w") as output_handle:
        # Writing original template DNA files 
        SeqIO.write(outputSeqMulti_DNA, output_handle, "fasta")
        # Writing orginal template RNA files 
        SeqIO.write(outputSeqMulti_convertedRNA, output_handle, "fasta")

# Not working or not being used 

In [ ]:
num_inF_TGG, num_inF_TTGG, num_inF_TGGA, num_inF_TTGGA, num_inF_ATG, num_inF_Stop, indices_inF_TGG, \
    indices_inF_ATG, indices_inF_Stop = \
    return_inFrame(Seq(seqTdT).reverse_complement(), 'all')

In [ ]:
num_inF_TGG

In [ ]:
num_inF_TTGG

In [ ]:
num_inF_TGGA

In [ ]:
num_inF_TTGGA

In [ ]:
# Trying to load entire transcripome ... obviously failed ... 
allRat_fileName = '/home/user1/Dropbox/Research/Neurobiology_PhD/Rotations/Huang/Projects/CellReadR/Data/Sequences/Reference/Ensembl/All/Rattus_norvegicus.Rnor_6.0.cdna.all.fa'
allRat = list(SeqIO.parse(allRat_fileName, "fasta"))